In [13]:
import pandas as pd
import numpy as np
# 读取数据环境基线_wang_3\环境基线_wang_3\环境基线_wang_3\Program\result\Matched_Data\2019\O3\安徽\matched_data_with_o3.csv
df = pd.read_csv(r'..\result\Matched_Data\2019\O3\上海\matched_data_with_o3.csv')
print(df.head())


  Station_Code Station_Name Province        Date  Mean_Pressure  \
0        1141A           普陀       上海  2019-01-01         1037.6   
1        1141A           普陀       上海  2019-01-02         1036.0   
2        1141A           普陀       上海  2019-01-03         1033.8   
3        1141A           普陀       上海  2019-01-04         1027.7   
4        1141A           普陀       上海  2019-01-05         1026.5   

   Mean_Humidity  Mean_Temperature  Mean_Wind_Speed  Max_Wind_Direction    O3  \
0           61.0               3.8              3.2                14.0  69.0   
1           62.0               4.6              2.4                 2.0  66.0   
2           67.0               7.0              1.6                 4.0  46.0   
3           89.0               9.6              2.0                 6.0  49.0   
4           95.0               8.1              2.3                 3.0  34.0   

   v1  
0  O3  
1  O3  
2  O3  
3  O3  
4  O3  


In [14]:
station_counts = df['Station_Code'].value_counts()
print(station_counts)

Station_Code
1141A    365
1142A    365
1143A    365
1144A    365
1145A    365
1147A    365
1148A    365
1149A    365
1150A    365
Name: count, dtype: int64


In [15]:

# 1. 找出数据最完整的站点（数据条数最多的）
station_counts = df['Station_Code'].value_counts()
max_count = station_counts.max()
valid_stations = station_counts[station_counts == max_count].index.tolist()

print(f"保留的站点数: {len(valid_stations)}")
# print(f"每个站点的数据天数: {max_count}")
df_valid = df[df['Station_Code'].isin(valid_stations)].copy()


保留的站点数: 9


In [18]:
# 6. 计算各省份各指标平均值
numeric_cols = ['Mean_Pressure', 'Mean_Humidity', 'Mean_Temperature', 
               'Mean_Wind_Speed', 'Max_Wind_Direction', 'O3']
result = df_valid.groupby(['Province', 'Date'])[numeric_cols].mean().reset_index()


print(result.head())


  Province        Date  Mean_Pressure  Mean_Humidity  Mean_Temperature  \
0       上海  2019-01-01         1037.6           61.0               3.8   
1       上海  2019-01-02         1036.0           62.0               4.6   
2       上海  2019-01-03         1033.8           67.0               7.0   
3       上海  2019-01-04         1027.7           89.0               9.6   
4       上海  2019-01-05         1026.5           95.0               8.1   

   Mean_Wind_Speed  Max_Wind_Direction         O3  
0              3.2                14.0  67.777778  
1              2.4                 2.0  67.777778  
2              1.6                 4.0  54.666667  
3              2.0                 6.0  46.888889  
4              2.3                 3.0  41.222222  


In [9]:

# 2. 筛选出数据完整的站点
df_valid = df[df['Station_Code'].isin(valid_stations)].copy()

# 3. 按省份和日期分组，计算平均值
result = df_valid.groupby(['Province', 'Date']).agg({
    'Mean_Pressure': 'mean',
    'Mean_Humidity': 'mean',
    'Mean_Temperature': 'mean',
    'Mean_Wind_Speed': 'mean',
    'Max_Wind_Direction': 'mean',
    'O3': 'mean',
    'human_effect': 'mean',
    'natural_effect': 'mean'
}).reset_index()

# 4. 添加时间相关列（从原始数据中保留）
time_cols = ['year', 'month', 'day', 'hour', 'weekday', 'dayofyear']
for col in time_cols:
    # 取每个日期第一次出现的值（所有站点同一天的值相同）
    date_values = df_valid.drop_duplicates('Date').set_index('Date')[col]
    result[col] = result['Date'].map(date_values)

# 5. 重新排列列顺序
cols_order = ['Province', 'Date'] + time_cols + [
    'Mean_Pressure', 'Mean_Humidity', 'Mean_Temperature',
    'Mean_Wind_Speed', 'Max_Wind_Direction', 'O3',
    'human_effect', 'natural_effect'
]
result = result[cols_order]
print(result.head())

  Province        Date  year  month  day  hour  weekday  dayofyear  \
0       上海  2019-01-01  2019      1    1     0        1          1   
1       上海  2019-01-02  2019      1    2     0        2          2   
2       上海  2019-01-03  2019      1    3     0        3          3   
3       上海  2019-01-04  2019      1    4     0        4          4   
4       上海  2019-01-05  2019      1    5     0        5          5   

   Mean_Pressure  Mean_Humidity  Mean_Temperature  Mean_Wind_Speed  \
0         1037.6           61.0               3.8              3.2   
1         1036.0           62.0               4.6              2.4   
2         1033.8           67.0               7.0              1.6   
3         1027.7           89.0               9.6              2.0   
4         1026.5           95.0               8.1              2.3   

   Max_Wind_Direction         O3  human_effect  natural_effect  
0                14.0  67.777778     67.601007        0.176771  
1                 2.0  67.77

In [ ]:

# 6. 保存结果
result.to_csv('province_daily_avg.csv', index=False)

print("处理完成！结果已保存为 province_daily_avg.csv")


In [17]:

# 打印站点数量
print(len(df['Station_Code'].unique()))

68


In [18]:

# 1. 筛选臭氧数据（假设 v1 列是污染物类型，O3 表示臭氧）
df_o3 = df[df["v1"] == "O3"].copy()

# 2. 找出所有日期范围（全局 min_date 和 max_date）
all_dates = pd.to_datetime(df_o3["Date"]).sort_values()
min_date, max_date = all_dates.min(), all_dates.max()
date_range = pd.date_range(min_date, max_date, freq="D")
date_columns = [d.strftime("%Y-%m-%d") for d in date_range]

# 3. 按站点分组，并检查数据长度是否匹配最长站点
max_length = len(date_range)
result = []

for station_code, group in df_o3.groupby("Station_Code"):
    # 检查该站点数据是否覆盖全部日期
    if len(group) == max_length:
        # 按日期排序并提取臭氧浓度
        group_sorted = group.sort_values("Date")
        o3_values = group_sorted.set_index("Date")["O3"].reindex(date_columns).values
        result.append([station_code] + o3_values.tolist())

# 4. 创建最终 DataFrame
final_df = pd.DataFrame(
    result,
    columns=["Station_Code"] + date_columns
)

# 5. 保存结果
print(final_df.head())
print(len(final_df))


  Station_Code  2019-01-01  2019-01-02  2019-01-03  2019-01-04  2019-01-05  \
0        1795A        56.0        55.0        36.0        17.0        18.0   
1        1798A        56.0        50.0        31.0        10.0        13.0   
2        1799A        55.0        55.0        36.0         8.0        10.0   
3        1800A        57.0        51.0        34.0        10.0        17.0   
4        1801A        35.0        47.0        30.0         6.0         8.0   

   2019-01-06  2019-01-07  2019-01-08  2019-01-09  ...  2019-12-22  \
0        30.0        14.0        36.0        34.0  ...        40.0   
1        24.0        16.0        41.0        34.0  ...        42.0   
2        30.0        19.0        49.0        39.0  ...        38.0   
3        24.0        25.0        37.0        34.0  ...        32.0   
4        22.0        16.0        37.0        28.0  ...        39.0   

   2019-12-23  2019-12-24  2019-12-25  2019-12-26  2019-12-27  2019-12-28  \
0        32.0        49.0        

In [30]:
# 1. 计算每个日期的平均值（按列计算）
daily_avg = final_df.drop(columns=["Station_Code"]).mean(axis=0)

# 2. 转换为 DataFrame，并添加 "省份" 列
result_df = pd.DataFrame({
    "省份": ["安徽"],  # 固定值
    **daily_avg.to_dict()  # 展开日期和平均值
})

# 3. 调整列顺序（省份列放第一列）
cols = ["省份"] + [col for col in result_df.columns if col != "省份"]
result_df = result_df[cols]

# 输出结果
print(result_df)

   省份  2019-01-01  2019-01-02  2019-01-03  2019-01-04  2019-01-05  2019-01-06  \
0  安徽   57.952472   55.021223   38.927183   27.856399   30.297242   43.807854   

   2019-01-07  2019-01-08  2019-01-09  ...  2019-12-22  2019-12-23  \
0   28.341721   42.573262   42.014106  ...   55.732294   36.286971   

   2019-12-24  2019-12-25  2019-12-26  2019-12-27  2019-12-28  2019-12-29  \
0   43.283509   22.838187   43.230074   64.087077    58.22315   55.382479   

   2019-12-30  2019-12-31  
0   44.076691   53.073229  

[1 rows x 366 columns]
